In [ ]:
import os
import pandas as pd
from typing import List, Dict
from sqlalchemy import create_engine, inspect

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama.llms import OllamaLLM
from langchain_core.documents import Document


/home/aliffatur/coding/kalbe-cdt-intern/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
DB_PATH = "bps_seki.db"

def load_all_tables(db_path: str) -> Dict[str, pd.DataFrame]:
    engine = create_engine(f"sqlite:///{db_path}")
    insp = inspect(engine)
    tables = {}
    for name in insp.get_table_names():
        df = pd.read_sql_table(name, engine)
        tables[name] = df
    return tables

print("Loading database ...")
tables = load_all_tables(DB_PATH)
print(f"Tables found: {list(tables.keys())}")

Loading database ...
Tables found: ['mkt_bps_angka_kelahiran', 'mkt_bps_gini_ratio', 'mkt_bps_inflasi_nasional', 'mkt_bps_jumlah_balita', 'mkt_bps_jumlah_ibu_hamil', 'mkt_bps_jumlah_penduduk', 'mkt_bps_jumlah_penduduk_per_usia', 'mkt_bps_jumlah_pns', 'mkt_bps_jumlah_tenaga_kesehatan', 'mkt_bps_pengeluaran_per_kapita', 'mkt_bps_persentase_bayi_asi_eksklusif', 'mkt_bps_produk_domestik_reg_bruto', 'mkt_bps_umr', 'ref_mkt_seki_devisa', 'ref_mkt_seki_exchange', 'ref_mkt_seki_export_import', 'ref_mkt_seki_ihk', 'ref_mkt_seki_indeks_harga', 'ref_mkt_seki_indonesia_ringkasan', 'ref_mkt_seki_inflasi', 'ref_mkt_seki_interest', 'ref_mkt_seki_investasi', 'ref_mkt_seki_pdb', 'ref_mkt_seki_savings', 'ref_mkt_seki_transaksi_berjalan_internasional']


In [3]:
def df_to_docs(name: str, df: pd.DataFrame) -> List[Document]:
    docs = []
    for i, row in df.iterrows():
        text = "\n".join([f"{col}: {row[col]}" for col in df.columns])
        docs.append(Document(page_content=text, metadata={"table": name, "row": i}))
    return docs

all_docs = []
for name, df in tables.items():
    all_docs.extend(df_to_docs(name, df))

print(f"Total documents created: {len(all_docs)}")

Total documents created: 113934


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama.llms import OllamaLLM
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, MessagesState, START, END
from operator import itemgetter

EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
OLLAMA_MODEL = "gemma3:latest"

#embedding & ingest to vectorstore database
emb = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
vectorstore = FAISS.from_documents(all_docs, emb)
print("Vectorstore built successfully!")

#ollama llm
llm = OllamaLLM(model=OLLAMA_MODEL)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

Vectorstore built successfully!
RetrievalQA pipeline (new version) ready!


In [19]:
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

prompt = ChatPromptTemplate.from_template("""
You are a knowledgeable assistant. Use the following context to answer the question.
If the answer cannot be found in the context, say "I'm not sure based on the provided information."

Context:
{context}

Question: {question}

Answer:
""")

# PERBAIKAN: Extract question string untuk retriever
qa_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x["question"])),  # ← Perbaikan
        "question": lambda x: x["question"],
    }
    | prompt
    | llm
)

In [20]:
from langgraph.graph import StateGraph, END
from typing import TypedDict


# --- State definition ---
class QAState(TypedDict):
    question: str
    answer: str
    continue_chat: bool

# --- Define graph ---
g = StateGraph(QAState)

# --- Node function ---
def ask(state: QAState):
    user_input = state["question"]
    
    # Jalankan chain dengan input
    result = qa_chain.invoke({"question": user_input})
    
    # Ambil hasil jawaban
    if isinstance(result, dict):
        answer = result.get("answer") or result.get("output_text") or str(result)
    else:
        answer = str(result)
    
    print("\n====================")
    print("Question:", user_input)
    print("\nAnswer:\n", answer)
    print("====================\n")
    
    return {"answer": answer, "continue_chat": False}

# Conditional edge function
def should_continue(state: QAState):
    return "ask" if state.get("continue_chat", False) else END

# Tambahkan node dan transisi
g.add_node("ask", ask)
g.set_entry_point("ask")
g.add_conditional_edges("ask", should_continue)

# Kompilasi graph
app = g.compile()

print("Chatbot ready! Type your question below (type 'exit' to quit).\n")

# Loop manual untuk interaksi
while True:
    try:
        user_question = input("Your question> ")
        
        if user_question.lower() in ['exit', 'quit', 'q']:
            print("\nSession ended.")
            break
        
        # Invoke graph dengan question
        result = app.invoke({"question": user_question, "answer": "", "continue_chat": False})
        
    except KeyboardInterrupt:
        print("\nSession ended.")
        break
    except Exception as e:
        print(f"\nError: {e}")
        continue

Chatbot ready! Type your question below (type 'exit' to quit).


Question: halo

Answer:
 I'm not sure based on the provided information.


Question: bisa jelaskan siapa kamu?

Answer:
 I am a knowledgeable assistant designed to answer questions based on the provided context.


Session ended.


In [ ]:
print(1)

In [21]:
# %%
import os
import pandas as pd
from typing import List, Dict, Tuple
from sqlalchemy import create_engine, inspect

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama.llms import OllamaLLM
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict

# %%
DB_PATH = "bps_seki.db"

def load_all_tables(db_path: str) -> Dict[str, pd.DataFrame]:
    """Load semua tabel dari database SQLite"""
    engine = create_engine(f"sqlite:///{db_path}")
    insp = inspect(engine)
    tables = {}
    for name in insp.get_table_names():
        df = pd.read_sql_table(name, engine)
        tables[name] = df
    return tables

print("Loading database ...")
tables = load_all_tables(DB_PATH)
print(f"Tables found: {list(tables.keys())}")

# %%
# IMPROVEMENT 1: Buat dokumen metadata untuk setiap tabel
def create_table_metadata_docs(tables: Dict[str, pd.DataFrame]) -> List[Document]:
    """
    Buat dokumen yang menjelaskan struktur dan isi setiap tabel.
    Ini membantu LLM memahami schema database.
    """
    docs = []
    
    for table_name, df in tables.items():
        # Informasi dasar tabel
        metadata = {
            "type": "table_metadata",
            "table": table_name,
            "row_count": len(df)
        }
        
        # Deskripsi tabel
        columns_info = []
        for col in df.columns:
            dtype = str(df[col].dtype)
            unique_count = df[col].nunique()
            
            # Ambil sample values (untuk kategorikal)
            if unique_count < 50:
                sample_values = df[col].unique()[:10].tolist()
                columns_info.append(
                    f"- {col} ({dtype}): {unique_count} unique values. "
                    f"Examples: {sample_values}"
                )
            else:
                columns_info.append(
                    f"- {col} ({dtype}): {unique_count} unique values"
                )
        
        content = f"""
Table: {table_name}
Description: BPS statistical data table
Row count: {len(df)}
Columns:
{chr(10).join(columns_info)}

This table contains BPS (Badan Pusat Statistik) data about {table_name.replace('_', ' ')}.
"""
        
        docs.append(Document(page_content=content, metadata=metadata))
    
    return docs

# IMPROVEMENT 2: Buat dokumen summary untuk setiap tabel
def create_table_summary_docs(tables: Dict[str, pd.DataFrame]) -> List[Document]:
    """
    Buat summary agregat untuk setiap tabel.
    Lebih efisien daripada per-row indexing.
    """
    docs = []
    
    for table_name, df in tables.items():
        # Identifikasi kolom kunci
        categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
        numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
        
        # Buat summary berdasarkan kategori
        if categorical_cols and len(categorical_cols) <= 3:
            # Group by kategori dan buat summary
            for group_cols in [categorical_cols[:2]] if len(categorical_cols) >= 2 else [categorical_cols[:1]]:
                try:
                    grouped = df.groupby(group_cols)
                    
                    for group_key, group_df in grouped:
                        if len(group_df) > 0:
                            # Buat deskripsi grup
                            if isinstance(group_key, tuple):
                                group_desc = ", ".join([f"{col}={val}" for col, val in zip(group_cols, group_key)])
                            else:
                                group_desc = f"{group_cols[0]}={group_key}"
                            
                            # Summary statistik untuk numeric columns
                            stats = []
                            for num_col in numeric_cols:
                                if num_col in group_df.columns:
                                    mean_val = group_df[num_col].mean()
                                    if pd.notna(mean_val):
                                        stats.append(f"{num_col} avg: {mean_val:.2f}")
                            
                            content = f"""
Table: {table_name}
Filter: {group_desc}
Records: {len(group_df)}
Statistics: {', '.join(stats) if stats else 'N/A'}

Data available for: {group_desc} in {table_name}
"""
                            
                            metadata = {
                                "type": "table_summary",
                                "table": table_name,
                                "filter": group_desc,
                                "row_count": len(group_df)
                            }
                            
                            docs.append(Document(page_content=content, metadata=metadata))
                except Exception as e:
                    print(f"Warning: Could not create summary for {table_name}: {e}")
    
    return docs

# IMPROVEMENT 3: Tetap buat dokumen per-row tapi lebih compact
def create_row_docs(tables: Dict[str, pd.DataFrame], sample_ratio: float = 0.3) -> List[Document]:
    """
    Buat dokumen per-row dengan format lebih compact.
    Sample hanya sebagian data untuk menghindari context overload.
    """
    docs = []
    
    for table_name, df in tables.items():
        # Sample data (opsional, hapus jika ingin semua data)
        if len(df) > 100:
            sampled_df = df.sample(n=int(len(df) * sample_ratio), random_state=42)
        else:
            sampled_df = df
        
        for idx, row in sampled_df.iterrows():
            # Format lebih compact: hanya kolom penting
            important_fields = []
            for col, val in row.items():
                if pd.notna(val) and val != '':
                    important_fields.append(f"{col}: {val}")
            
            content = f"Table: {table_name}\n" + " | ".join(important_fields[:10])  # Limit 10 fields
            
            metadata = {
                "type": "row_data",
                "table": table_name,
                "row_index": idx
            }
            
            docs.append(Document(page_content=content, metadata=metadata))
    
    return docs

# Buat semua dokumen
print("\nCreating documents...")
metadata_docs = create_table_metadata_docs(tables)
summary_docs = create_table_summary_docs(tables)
row_docs = create_row_docs(tables, sample_ratio=0.5)  # 50% sampling

all_docs = metadata_docs + summary_docs + row_docs

print(f"Total documents created:")
print(f"  - Metadata docs: {len(metadata_docs)}")
print(f"  - Summary docs: {len(summary_docs)}")
print(f"  - Row docs: {len(row_docs)}")
print(f"  - TOTAL: {len(all_docs)}")

# %%
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
OLLAMA_MODEL = "gemma3:latest"

# Embedding & ingest to vectorstore database
print("\nBuilding vectorstore...")
emb = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
vectorstore = FAISS.from_documents(all_docs, emb)
print("Vectorstore built successfully!")

# Ollama LLM
llm = OllamaLLM(model=OLLAMA_MODEL)

# IMPROVEMENT 4: Retriever dengan filter metadata
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}  # Ambil lebih banyak untuk filtering
)

# %%
# IMPROVEMENT 5: Enhanced formatting dengan prioritas metadata
def format_docs(docs):
    """Format dokumen dengan prioritas: metadata > summary > row data"""
    # Pisahkan berdasarkan tipe
    metadata_docs = [d for d in docs if d.metadata.get("type") == "table_metadata"]
    summary_docs = [d for d in docs if d.metadata.get("type") == "table_summary"]
    row_docs = [d for d in docs if d.metadata.get("type") == "row_data"]
    
    formatted = []
    
    # Prioritaskan metadata
    if metadata_docs:
        formatted.append("=== TABLE SCHEMAS ===")
        formatted.extend([d.page_content for d in metadata_docs[:2]])
    
    # Lalu summary
    if summary_docs:
        formatted.append("\n=== DATA SUMMARIES ===")
        formatted.extend([d.page_content for d in summary_docs[:5]])
    
    # Terakhir row data
    if row_docs:
        formatted.append("\n=== SAMPLE DATA ===")
        formatted.extend([d.page_content for d in row_docs[:5]])
    
    return "\n\n".join(formatted)

# IMPROVEMENT 6: Enhanced prompt dengan instruksi lebih jelas
prompt = ChatPromptTemplate.from_template("""
You are a knowledgeable assistant specialized in Indonesian BPS (Badan Pusat Statistik) data.

Use the following context to answer the question accurately. The context includes:
1. Table schemas (structure and column information)
2. Data summaries (aggregated statistics)
3. Sample data (actual records)

IMPORTANT INSTRUCTIONS:
- If the answer involves numbers, provide specific values from the context
- If you need to compare data across years, regions, or categories, use the summaries
- If the context doesn't contain enough information, say "I don't have enough data to answer this accurately"
- Always mention which table(s) the information comes from
- Format numbers clearly (use thousands separators when appropriate)

Context:
{context}

Question: {question}

Answer (in Indonesian):
""")

# QA Chain
qa_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x["question"])),
        "question": lambda x: x["question"],
    }
    | prompt
    | llm
)

print("Enhanced QA chain ready!")

# %%
# IMPROVEMENT 7: Better state management dengan history
class QAState(TypedDict):
    question: str
    answer: str
    continue_chat: bool
    history: List[str]  # Track conversation history

g = StateGraph(QAState)

def ask(state: QAState):
    user_input = state["question"]
    
    # Jalankan chain
    result = qa_chain.invoke({"question": user_input})
    
    # Extract answer
    if isinstance(result, dict):
        answer = result.get("answer") or result.get("output_text") or str(result)
    else:
        answer = str(result)
    
    print("\n" + "="*50)
    print("❓ Question:", user_input)
    print("\n💡 Answer:")
    print(answer)
    print("="*50 + "\n")
    
    # Update history
    history = state.get("history", [])
    history.append(f"Q: {user_input}\nA: {answer}")
    
    return {
        "answer": answer,
        "continue_chat": False,
        "history": history
    }

def should_continue(state: QAState):
    return "ask" if state.get("continue_chat", False) else END

g.add_node("ask", ask)
g.set_entry_point("ask")
g.add_conditional_edges("ask", should_continue)

app = g.compile()

print("🤖 BPS Chatbot Ready!")
print("="*50)
print("Commands:")
print("  - Type your question in Indonesian or English")
print("  - Type 'exit', 'quit', or 'q' to quit")
print("="*50 + "\n")

# Main loop
while True:
    try:
        user_question = input("Your question> ")
        
        if user_question.lower().strip() in ['exit', 'quit', 'q', '']:
            print("\n👋 Session ended. Thank you!")
            break
        
        result = app.invoke({
            "question": user_question,
            "answer": "",
            "continue_chat": False,
            "history": []
        })
        
    except KeyboardInterrupt:
        print("\n\n👋 Session ended. Thank you!")
        break
    except Exception as e:
        print(f"\n❌ Error: {e}\n")
        continue

# %%
print("✅ Script completed!")

Loading database ...
Tables found: ['mkt_bps_angka_kelahiran', 'mkt_bps_gini_ratio', 'mkt_bps_inflasi_nasional', 'mkt_bps_jumlah_balita', 'mkt_bps_jumlah_ibu_hamil', 'mkt_bps_jumlah_penduduk', 'mkt_bps_jumlah_penduduk_per_usia', 'mkt_bps_jumlah_pns', 'mkt_bps_jumlah_tenaga_kesehatan', 'mkt_bps_pengeluaran_per_kapita', 'mkt_bps_persentase_bayi_asi_eksklusif', 'mkt_bps_produk_domestik_reg_bruto', 'mkt_bps_umr', 'ref_mkt_seki_devisa', 'ref_mkt_seki_exchange', 'ref_mkt_seki_export_import', 'ref_mkt_seki_ihk', 'ref_mkt_seki_indeks_harga', 'ref_mkt_seki_indonesia_ringkasan', 'ref_mkt_seki_inflasi', 'ref_mkt_seki_interest', 'ref_mkt_seki_investasi', 'ref_mkt_seki_pdb', 'ref_mkt_seki_savings', 'ref_mkt_seki_transaksi_berjalan_internasional']

Creating documents...
Total documents created:
  - Metadata docs: 25
  - Summary docs: 2658
  - Row docs: 56964
  - TOTAL: 59647

Building vectorstore...
Vectorstore built successfully!
Enhanced QA chain ready!
🤖 BPS Chatbot Ready!
Commands:
  - Type your